### Setup

In [1]:
import datetime

import dask.dataframe as dd
import geopandas as gpd
import matplotlib.pyplot as plt
import movingpandas as mpd
import pandas as pd

c:\Users\edgar\anaconda3\envs\movingpandas\lib\site-packages\movingpandas\__init__.py:37: UserWarning: Missing optional dependencies. To use the trajectory smoother classes please install Stone Soup (see https://stonesoup.readthedocs.io/en/latest/#installation).
  warnings.warn(e.msg, UserWarning)


In [2]:
def load_InTAS(path: str) -> pd.DataFrame:
    intas = dd.read_csv(
        path,
        sep=";",
        usecols=[
            "timestep_time",
            "vehicle_id",
            "vehicle_x",
            "vehicle_y",
            "person_id",
            "person_x",
            "person_y",
        ],
    )
    intas['trajectory_id'] = intas["vehicle_id"].fillna(intas["person_id"])
    intas['x'] = intas["vehicle_x"].fillna(intas["person_x"])
    intas['y'] = intas["vehicle_y"].fillna(intas["person_y"])
    InTAS_datetime = datetime.datetime(2019, 11, 1)
    intas["t"] = dd.to_datetime(
        intas["timestep_time"],
        unit="s",
        origin=InTAS_datetime,
    )
    intas = intas.drop(columns=[
        'vehicle_id',
        'vehicle_x',
        'vehicle_y',
        'person_id',
        'person_x',
        'person_y',
    ])

    return intas.compute()

In [3]:
def hist_plot(data1, data2, title1, title2, xlabel, ylabel):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))

    # Plotar o primeiro histograma no primeiro subplot
    axes[0].hist(data1, bins=30, color='blue', alpha=0.7)
    axes[0].set_title(title1)
    axes[0].set_xlabel(xlabel)
    axes[0].set_ylabel(ylabel)

    # Plotar o segundo histograma no segundo subplot
    axes[1].hist(data2, bins=30, color='green', alpha=0.7)
    axes[1].set_title(title2)
    axes[1].set_xlabel(xlabel)
    axes[1].set_ylabel(ylabel)

    # Ajustar o layout para evitar sobreposição
    plt.tight_layout()

    # Mostrar os plots
    plt.show()

In [4]:
intas_title1 = "InTAS (6-10 horas)"
df1 = load_InTAS("./datasets/6_as_10hrs/fcd.csv")
intas1 = mpd.TrajectoryCollection(
    df1,
    "trajectory_id",
    x="x",
    y="y",
    t="t",
)

In [5]:
intas_title2 = "InTAS (15-19 horas)"
df2 = load_InTAS("./datasets/15_as_19hrs/fcd.csv")
intas2 = mpd.TrajectoryCollection(
    df2,
    "trajectory_id",
    x="x",
    y="y",
    t="t",
)

### Waiting Times (Point-to-Point)

In [6]:
%%time
intas1.add_timedelta()
intas2.add_timedelta()

Wall time: 2min 5s


TrajectoryCollection with 26407 trajectories

### Distance (Point-to-Point)

In [7]:
%%time
intas1.add_distance()
intas2.add_distance()

Wall time: 41min 6s


TrajectoryCollection with 26407 trajectories

### Speed (Point-to-Point)

In [8]:
%%time
intas1.add_speed()
intas2.add_speed()

Wall time: 44min 32s


TrajectoryCollection with 26407 trajectories

### Turning Angle (Point-to-Point)

In [9]:
%%time
intas1.add_direction()
intas2.add_direction()

Wall time: 22min 6s


TrajectoryCollection with 26407 trajectories

### Compression

In [10]:
%%time
generalized1 = mpd.MinDistanceGeneralizer(intas1).generalize(tolerance=100)
generalized2 = mpd.MinDistanceGeneralizer(intas2).generalize(tolerance=100)

Wall time: 49min 44s


### Clustering

In [20]:
%%time
aggregator1 = mpd.TrajectoryCollectionAggregator(
    generalized1,
    max_distance=1000,
    min_distance=100,
    min_stop_duration=datetime.timedelta(minutes=5),
)
aggregator2 = mpd.TrajectoryCollectionAggregator(
    generalized2,
    max_distance=1000,
    min_distance=100,
    min_stop_duration=datetime.timedelta(minutes=5),
)

c:\Users\edgar\anaconda3\envs\movingpandas\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


AttributeError: module 'statistics' has no attribute 'fmean'

### Radius of Gyration (N/A)

### Distance straight line (N/A)

### Stay Locations

In [12]:
%%time
detector1 = mpd.TrajectoryStopDetector(intas1)
detector2 = mpd.TrajectoryStopDetector(intas2)

Wall time: 997 µs


In [13]:
from datetime import timedelta

In [ ]:
%%time
stop_points1 = detector1.get_stop_points(
    min_duration=timedelta(seconds=60), max_diameter=100
)
stop_points2 = detector2.get_stop_points(
    min_duration=timedelta(seconds=60), max_diameter=100
)

### Home e Work Classification (N/A)

### Outliers Detection

In [15]:
%%time
clean1 = mpd.OutlierCleaner(intas1).clean(v_max=500, units=("km", "h"))
clean2 = mpd.OutlierCleaner(intas2).clean(v_max=500, units=("km", "h"))

Wall time: 1h 23min 10s


### Spatial Projection

In [ ]:
%%time
plot1 = intas1.hvplot(
    title=intas_title1,
    rasterize=True,
    dynspread=True,
    colorbar=True,
    cnorm="eq_hist",
    cmap="inferno",
    width=650,
    height=500,
)
plot1 = intas2.hvplot(
    title=intas_title1,
    rasterize=True,
    dynspread=True,
    colorbar=True,
    cnorm="eq_hist",
    cmap="inferno",
    width=650,
    height=500,
)